# resnet50 多个超参数网格搜索 看哪个效果好


Transfer Learning for Computer Vision Tutorial
==============================================

These two major transfer learning scenarios look as follows:

-  **Finetuning the convnet**: Instead of random initialization, we
   initialize the network with a pretrained network, like the one that is
   trained on imagenet 1000 dataset. Rest of the training looks as
   usual.
-  **ConvNet as fixed feature extractor**: Here, we will freeze the weights
   for all of the network except that of the final fully connected
   layer. This last fully connected layer is replaced with a new one
   with random weights and only this layer is trained.


In [7]:
# from google.colab import drive
# drive.mount('/content/drive')

In [8]:
# %matplotlib inline

In [9]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode
# 待搜索超参数
LR_LIB = [3e-4,1e-4,1e-5]
BS_LIB = [8,16,32,64]
DO_lib = [0.3,0.4,0.5,0.6,0.7]
bestacc = np.zeros([len(LR_LIB),len(BS_LIB),len(DO_lib)])

In [10]:
def train_and_valid(model, loss_function, optimizer, epochs=25):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    history = []
    best_acc = 0.0
    best_epoch = 0

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))

        model.train()

        train_loss = 0.0
        train_acc = 0.0
        valid_loss = 0.0
        valid_acc = 0.0

        for i, (inputs, labels) in enumerate(train_data):
            inputs = inputs.to(device)
            labels = labels.to(device)

            #因为这里梯度是累加的，所以每次记得清零
            optimizer.zero_grad()

            outputs = model(inputs)

            loss = loss_function(outputs, labels)

            loss.backward()

            optimizer.step()

            train_loss += loss.item() * inputs.size(0)

            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))

            acc = torch.mean(correct_counts.type(torch.FloatTensor))

            train_acc += acc.item() * inputs.size(0)
            
#             del inputs,labels,outputs,loss
#             torch.cuda.empty_cache()

        with torch.no_grad():
            model.eval()

            for j, (inputs, labels) in enumerate(valid_data):
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)

                loss = loss_function(outputs, labels)

                valid_loss += loss.item() * inputs.size(0)

                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                valid_acc += acc.item() * inputs.size(0)
                
#                 del inputs,labels,outputs,loss
#                 torch.cuda.empty_cache()

        avg_train_loss = train_loss/train_data_size
        avg_train_acc = train_acc/train_data_size

        avg_valid_loss = valid_loss/valid_data_size
        avg_valid_acc = valid_acc/valid_data_size

        history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])

        if best_acc < avg_valid_acc:
            best_acc = avg_valid_acc
            best_epoch = epoch + 1

        epoch_end = time.time()

        print("Epoch: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation: Loss: {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(
            epoch+1, avg_valid_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start
        ))
        print("Best Accuracy for validation : {:.4f} at epoch {:03d}".format(best_acc, best_epoch))
        dataset1 = 'E:\gmd_data\microsphere2_dataAugmented'
        torch.save(model, dataset1+'/models/'+'model_'+str(epoch+1)+'.pt')
#         torch.save(model, 'models/'+dataset+'_model_'+str(epoch+1)+'.pt')
        if epoch + 1 - 100 >  best_epoch:
            
            break
    return model, history, best_acc


In [11]:
def grid_search(ii,jj,kk):
    # Data augmentation and normalization for training
    # Just normalization for validation
    image_transforms  = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
            transforms.RandomRotation(degrees=45),
            transforms.RandomHorizontalFlip(),
            transforms.Resize(size=224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'valid': transforms.Compose([
            transforms.Resize(size=256),
            transforms.CenterCrop(size=224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    }

    dataset = 'microsphere2'
    train_directory = os.path.join(dataset, 'train')
    valid_directory = os.path.join(dataset, 'valid')

    global LR_LIB
    global BS_LIB
    global DO_lib
    Lr = LR_LIB[ii]
    batch_size = BS_LIB[jj]
    Dr = DO_lib[kk]

    # num_classes = 173

    data = {
        'train': datasets.ImageFolder(root=train_directory, transform=image_transforms['train']),
        'valid': datasets.ImageFolder(root=valid_directory, transform=image_transforms['valid'])

    }


    global train_data_size
    global valid_data_size
    train_data_size = len(data['train'])
    valid_data_size = len(data['valid'])

    global train_data
    global valid_data
    train_data = DataLoader(data['train'], batch_size=batch_size, shuffle=True)
    valid_data = DataLoader(data['valid'], batch_size=batch_size, shuffle=True)

    print(train_data_size, valid_data_size,"LR: {:.4f} bs: {:03d} DO: {:.4f}".format(Lr, batch_size,Dr))

    #迁移学习
    #这里使用ResNet-50的预训练模型。

    resnet50 = models.resnet50(pretrained=True)

    for param in resnet50.parameters():
        param.requires_grad = True

    fc_inputs = resnet50.fc.in_features
    resnet50.fc = nn.Sequential(
        nn.Linear(fc_inputs, 256),
        nn.ReLU(),
        nn.Dropout(Dr),
        nn.Linear(256,  173),
        nn.LogSoftmax(dim = 1)


    )
    resnet50 = resnet50.to('cuda:0')
    loss_func = nn.NLLLoss()
    optimizer = optim.AdamW(resnet50.parameters(),lr=Lr)
    
    # Training the model
    num_epochs = 3000

    global bestacc
    trained_model, history, bestacc[ii,jj,kk] = train_and_valid(resnet50, loss_func, optimizer, num_epochs)
    torch.save(history, 'models/'+dataset+'_history.pt')
    return 0

In [12]:
for ii in range(1,len(LR_LIB)):
    for jj in range(len(BS_LIB)):
        for kk in range(len(DO_lib)):
            grid_search(ii,jj,kk)
#             del LR_LIB,BS_LIB,DO_lib,train_data_size,valid_data_size,train_data,valid_data
#             torch.cuda.empty_cache()

1038 346 LR: 0.0001 bs: 008 DO: 0.3000
Epoch: 1/3000
Epoch: 001, Training: Loss: 5.0986, Accuracy: 0.1927%, 
		Validation: Loss: 5.0986, Accuracy: 1.7341%, Time: 17.0174s
Best Accuracy for validation : 0.0173 at epoch 001
Epoch: 2/3000
Epoch: 002, Training: Loss: 4.8690, Accuracy: 0.8671%, 
		Validation: Loss: 4.8690, Accuracy: 1.4451%, Time: 16.8632s
Best Accuracy for validation : 0.0173 at epoch 001
Epoch: 3/3000
Epoch: 003, Training: Loss: 4.5841, Accuracy: 0.9634%, 
		Validation: Loss: 4.5841, Accuracy: 1.7341%, Time: 16.6184s
Best Accuracy for validation : 0.0173 at epoch 001
Epoch: 4/3000
Epoch: 004, Training: Loss: 4.3774, Accuracy: 1.9268%, 
		Validation: Loss: 4.3774, Accuracy: 1.7341%, Time: 16.7085s
Best Accuracy for validation : 0.0173 at epoch 001
Epoch: 5/3000
Epoch: 005, Training: Loss: 4.2825, Accuracy: 1.5414%, 
		Validation: Loss: 4.2825, Accuracy: 2.8902%, Time: 16.7030s
Best Accuracy for validation : 0.0289 at epoch 005
Epoch: 6/3000
Epoch: 006, Training: Loss: 4.25

In [18]:
bestacc
# history = np.array(history)
# plt.plot(history[:, 0:2])
# plt.legend(['Tr Loss', 'Val Loss'])
# plt.xlabel('Epoch Number')
# plt.ylabel('Loss')
# plt.ylim(0, 1)
# plt.savefig(dataset+'_loss_curve.png')
# plt.show()

# plt.plot(history[:, 2:4])
# plt.legend(['Tr Accuracy', 'Val Accuracy'])
# plt.xlabel('Epoch Number')
# plt.ylabel('Accuracy')
# plt.ylim(0, 1)
# plt.savefig(dataset+'_accuracy_curve.png')
# plt.show()

array([[[0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ]],

       [[0.4017341 , 0.41040462, 0.40462428, 0.44219653, 0.34682081],
        [0.35260116, 0.38439306, 0.38439306, 0.43930636, 0.45953757],
        [0.34393064, 0.36127168, 0.39306358, 0.39884393, 0.4132948 ],
        [0.38150289, 0.37861272, 0.38439306, 0.39884393, 0.42196532]],

       [[0.38728324, 0.4132948 , 0.43352601, 0.43063584, 0.43930636],
        [0.41907515, 0.42196532, 0.42485549, 0.39884393, 0.39595376],
        [0.44508671, 0.41907515, 0.37861272, 0.41907514, 0.34971098],
        [0.39595376, 0.41040462, 0.28612717, 0.39017341, 0.26589595]]])

In [14]:
# import numpy as np
# LR_LIB = [1,1e-1,1e-2,1e-3,3e-4,1e-4,1e-5]
# BS_LIB = [8,16,32,64,128,256]
# DO_lib = [0.3,0.4,0.5,0.6,0.7]
# bestacc = np.zeros([len(LR_LIB),len(BS_LIB),len(DO_lib)])
# bestacc[2,3,1] = 1
# bestacc
ii

2

In [15]:
del LR_LIB,BS_LIB,DO_lib,train_data_size,valid_data_size,train_data,valid_data
torch.cuda.empty_cache()

In [16]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    1027 MB |    8967 MB |    6678 TB |    6678 TB |
|       from large pool |     950 MB |    8872 MB |    6614 TB |    6614 TB |
|       from small pool |      77 MB |     156 MB |      64 TB |      64 TB |
|---------------------------------------------------------------------------|
| Active memory         |    1027 MB |    8967 MB |    6678 TB |    6678 TB |
|       from large pool |     950 MB |    8872 MB |    6614 TB |

In [17]:
len(BS_LIB)

NameError: name 'BS_LIB' is not defined

In [ ]:
for iii in range(4,len(BS_LIB)):
    print(iii)

In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
torch.cuda.empty_cache()

In [ ]:
pip install jupyter_contrib_nbextensions
jupyter contrib nbextension install --user